In [1]:
# Imports

import os

import csv

from selenium import webdriver
from requests import get
from bs4 import BeautifulSoup

In [19]:
# Statics
# Note: this URL works, but could probably be cleaned up

base_url = "http://bobcat.library.nyu.edu/primo_library/libweb/action/"
base_url += "search.do?fn=search&ct=search&initialSearch=true&mode=Basic&tab=all&indx=1&dum=true&srt=rank&vid=NYU&frbg=&vl%28freeText0%29="

infile = "data/isbns.txt"

In [20]:
# Read a txt file of isbns

with open(infile, "r") as f:
    isbns = f.read().splitlines()

print(isbns)
#test_isbns = ['9781848825543']

['9782711857319', '9782754106207', '1588390721', '9782842782177', '9782855394237', '2855394236', '088029177x', '9785938930506', '9781891771255', '9780910407397', '9780939117673', '9780911886535', '299970701', '810929384', '9789748225302', '9780761816713', '9780140443486', '9780140443066', '9780192835765', '942614186', '9780195659122', '8190014900', '773478531', '2720010316', '9780862671501', '9782070532551', '376302011x', '8121507235', '9783869221878', '9780520224667', '932216285', '7800070433', '7226qq4976', '7226004976', '3201016780', '9780553370904', '877733546', '9782711823642', '2855397359', '9781851774296', '7560710077', '9787228032488', '9782220026299', '9780714121802', '9780813380650', '9780810981034', '067040697x', '9781904832164', '9780907849520', '9780521533386', '9789748225302', '9748434575', '500690014', '9780811800402', '3761106262', '9782842782177', '520050614', '9780252071218', '810906813', '9781570621864', '9780500275160', '9782080113603', '029596815x', '934686505', '6

In [30]:
def validate_isbn(isbn):
    if len(isbn) == 13 or len(isbn) == 10:
        pass
    else:
        isbn = '0' * (10-len(isbn)) + isbn
    return isbn

In [32]:
# Fix leading zeroes
# Should be other validation?

isbns = [validate_isbn(isbn) for isbn in isbns]

In [33]:
# Create browser instance
# Note: this requires Firefox to be installed

browser = webdriver.Firefox()

In [34]:
def check_bobcat_isbn(isbn):
    check_bobcat = False # Set default return
    url = base_url + isbn # Build URL string
    browser.get(url) # Open url in browser instance; should trap response errors
    alert = browser.find_elements_by_class_name('alert') # Only missing ISBNs have an class called alert
    if len(alert) == 0:
        check_bobcat = True
    return check_bobcat

In [36]:
matches = []

for isbn in isbns:
    if check_bobcat_isbn(isbn):
        print("Yes! %s is in the NYU catalog" % isbn)
        matches.append((isbn, True))
    else:
        print("No! %s is not in the NYU catalog" % isbn)
        matches.append((isbn, False))

Yes! 9782711857319 is in the NYU catalog
No! 9782754106207 is not in the NYU catalog
No! 1588390721 is not in the NYU catalog
No! 9782842782177 is not in the NYU catalog
No! 9782855394237 is not in the NYU catalog
No! 2855394236 is not in the NYU catalog
No! 088029177x is not in the NYU catalog
No! 9785938930506 is not in the NYU catalog
Yes! 9781891771255 is in the NYU catalog
Yes! 9780910407397 is in the NYU catalog
Yes! 9780939117673 is in the NYU catalog
Yes! 9780911886535 is in the NYU catalog
Yes! 0299970701 is in the NYU catalog
No! 0810929384 is not in the NYU catalog
Yes! 9789748225302 is in the NYU catalog
Yes! 9780761816713 is in the NYU catalog
Yes! 9780140443486 is in the NYU catalog
Yes! 9780140443066 is in the NYU catalog
No! 9780192835765 is not in the NYU catalog
Yes! 0942614186 is in the NYU catalog
No! 9780195659122 is not in the NYU catalog
No! 8190014900 is not in the NYU catalog
Yes! 0773478531 is in the NYU catalog
Yes! 2720010316 is in the NYU catalog
No! 978086

In [37]:
# Export to csv

with open('matches.csv','w') as out:
    csv_out=csv.writer(out, quotechar = "'")
    csv_out.writerow(['isbn','match'])
    for row in matches:
        csv_out.writerow(row)

In [38]:
# Close browser instance
browser.quit()